In [2]:
import pandas as pd
import os
import requests
import gc
import torch
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
import numpy as np
import shutil
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
gc.collect()

20

In [4]:
df = pd.read_csv('train-0-en.tsv', sep='\t')
df = df[~df.mime_type.isin(['image/svg+xml','image/webp'])]

In [5]:
gc.collect()

0

In [6]:
df

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
0,en,https://en.wikipedia.org/wiki/Oxydactylus,https://upload.wikimedia.org/wikipedia/commons...,Oxydactylus,NaN,Oxydactylus,NaN,English: Mounted skeleton of Oxydactylus longi...,NaN,image/jpeg,3564,2748,True,True,False,Oxydactylus is an extinct genus of camelid end...,Oxydactylus is an extinct genus of camelid end...
1,en,https://en.wikipedia.org/wiki/Maine%27s_3rd_co...,https://upload.wikimedia.org/wikipedia/commons...,Maine's 3rd congressional district,List of members representing the district,Maine's 3rd congressional district / List of m...,NaN,"English: Samuel W. Gould, US Representative fr...",NaN,image/jpeg,1870,1421,False,True,False,Maine's 3rd congressional district is an obsol...,NaN
2,en,https://en.wikipedia.org/wiki/Cheraw,https://upload.wikimedia.org/wikipedia/commons...,Cheraw,18th century,Cheraw / History / 18th century,A c. 1724 English copy of a deerskin Catawba m...,"English: ""Map of the Several Nations of Indian...",NaN,image/jpeg,1217,1770,False,True,False,"The Cheraw people, also known as the Saraw or ...","In 1710, due to attacks by the Seneca of the I..."
3,en,https://en.wikipedia.org/wiki/Romanian_Front,https://upload.wikimedia.org/wikipedia/commons...,Romanian Front,Stagnation,Romanian Front / History / Stagnation,"Nameplate of Gazeta Transilvaniei on June 14, ...",English: Nameplate of the Romanian nationalist...,NaN,image/png,306,960,False,True,True,The Romanian Front was a moderate fascist part...,A reshuffled Tătărescu government took over in...
4,en,https://en.wikipedia.org/wiki/%C3%81d%C3%A1m_K...,https://upload.wikimedia.org/wikipedia/commons...,Ádám Kósa,NaN,Ádám Kósa,NaN,English: Hungarian MEP Ádám Kósa,NaN,image/jpeg,4928,3264,True,False,True,Ádám Kósa is a Hungarian politician and Member...,Ádám Kósa (born 1 July 1975) is a Hungarian po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542588,en,https://en.wikipedia.org/wiki/History_of_Latin,https://upload.wikimedia.org/wikipedia/commons...,History of Latin,Old Latin,History of Latin / Old Latin,"The Duenos inscription, from the 6th century B...",NaN,NaN,image/jpeg,464,448,False,False,True,Latin is a member of the broad family of Itali...,Old Latin (also called Early Latin or Archaic ...
542589,en,https://en.wikipedia.org/wiki/Battenberg_Castle,http://upload.wikimedia.org/wikipedia/commons/...,Battenberg Castle,NaN,Battenberg Castle,The Blitzröhren below the castle ruins,English: The so-called lightning tubes near Ba...,NaN,image/jpeg,1704,2272,False,True,False,Battenberg Castle is a castle ruin near Batten...,Battenberg Castle (German: Burg Battenberg) is...
542590,en,https://en.wikipedia.org/wiki/List_of_research...,https://upload.wikimedia.org/wikipedia/commons...,List of research vessels by country,National Oceanic and Atmospheric Administration,List of research vessels by country / United S...,"NOAAS Rainier with her namesake, Mount Rainier...",English: NOAAS Rainier (S 221) with Mount Rain...,NaN,image/jpeg,237,400,False,True,True,Following is a list of research vessels by cou...,NOAAS Bell M. Shimada\nNOAAS David Starr Jorda...
542591,en,https://en.wikipedia.org/wiki/Kino_Works,https://upload.wikimedia.org/wikipedia/commons...,Kino Works,NaN,Kino Works,NaN,English: Kino Works LLP Official Logo,NaN,image/jpeg,400,400,True,True,True,Kino Works LLP is an India based production ho...,Kino Works LLP is an India based production ho...


In [7]:
df = df.reset_index(drop=True)

In [8]:
df = df.reset_index()

In [9]:
df = df.rename(columns={'index': 'index'})

In [10]:
df

,index,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
0,0,en,https://en.wikipedia.org/wiki/Oxydactylus,https://upload.wikimedia.org/wikipedia/commons...,Oxydactylus,NaN,Oxydactylus,NaN,English: Mounted skeleton of Oxydactylus longi...,NaN,image/jpeg,3564,2748,True,True,False,Oxydactylus is an extinct genus of camelid end...,Oxydactylus is an extinct genus of camelid end...
1,1,en,https://en.wikipedia.org/wiki/Maine%27s_3rd_co...,https://upload.wikimedia.org/wikipedia/commons...,Maine's 3rd congressional district,List of members representing the district,Maine's 3rd congressional district / List of m...,NaN,"English: Samuel W. Gould, US Representative fr...",NaN,image/jpeg,1870,1421,False,True,False,Maine's 3rd congressional district is an obsol...,NaN
2,2,en,https://en.wikipedia.org/wiki/Cheraw,https://upload.wikimedia.org/wikipedia/commons...,Cheraw,18th century,Cheraw / History / 18th century,A c. 1724 English copy of a deerskin Catawba m...,"English: ""Map of the Several Nations of Indian...",NaN,image/jpeg,1217,1770,False,True,False,"The Cheraw people, also known as the Saraw or ...","In 1710, due to attacks by the Seneca of the I..."
3,3,en,https://en.wikipedia.org/wiki/Romanian_Front,https://upload.wikimedia.org/wikipedia/commons...,Romanian Front,Stagnation,Romanian Front / History / Stagnation,"Nameplate of Gazeta Transilvaniei on June 14, ...",English: Nameplate of the Romanian nationalist...,NaN,image/png,306,960,False,True,True,The Romanian Front was a moderate fascist part...,A reshuffled Tătărescu government took over in...
4,4,en,https://en.wikipedia.org/wiki/%C3%81d%C3%A1m_K...,https://upload.wikimedia.org/wikipedia/commons...,Ádám Kósa,NaN,Ádám Kósa,NaN,English: Hungarian MEP Ádám Kósa,NaN,image/jpeg,4928,3264,True,False,True,Ádám Kósa is a Hungarian politician and Member...,Ádám Kósa (born 1 July 1975) is a Hungarian po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525673,525673,en,https://en.wikipedia.org/wiki/History_of_Latin,https://upload.wikimedia.org/wikipedia/commons...,History of Latin,Old Latin,History of Latin / Old Latin,"The Duenos inscription, from the 6th century B...",NaN,NaN,image/jpeg,464,448,False,False,True,Latin is a member of the broad family of Itali...,Old Latin (also called Early Latin or Archaic ...
525674,525674,en,https://en.wikipedia.org/wiki/Battenberg_Castle,http://upload.wikimedia.org/wikipedia/commons/...,Battenberg Castle,NaN,Battenberg Castle,The Blitzröhren below the castle ruins,English: The so-called lightning tubes near Ba...,NaN,image/jpeg,1704,2272,False,True,False,Battenberg Castle is a castle ruin near Batten...,Battenberg Castle (German: Burg Battenberg) is...
525675,525675,en,https://en.wikipedia.org/wiki/List_of_research...,https://upload.wikimedia.org/wikipedia/commons...,List of research vessels by country,National Oceanic and Atmospheric Administration,List of research vessels by country / United S...,"NOAAS Rainier with her namesake, Mount Rainier...",English: NOAAS Rainier (S 221) with Mount Rain...,NaN,image/jpeg,237,400,False,True,True,Following is a list of research vessels by cou...,NOAAS Bell M. Shimada\nNOAAS David Starr Jorda...
525676,525676,en,https://en.wikipedia.org/wiki/Kino_Works,https://upload.wikimedia.org/wikipedia/commons...,Kino Works,NaN,Kino Works,NaN,English: Kino Works LLP Official Logo,NaN,image/jpeg,400,400,True,True,True,Kino Works LLP is an India based production ho...,Kino Works LLP is an India based production ho...


In [11]:
folder_path = 'images/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

model_path = './vit-base-beans'
model_name = 'ViTForImageClassification'
test_folder = './images'

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = ViTForImageClassification.from_pretrained(model_path).to(device)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_path, device=device)

/opt/homebrew/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [12]:
def transform(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = feature_extractor(images=image.convert('RGB'), return_tensors='pt')
    return inputs.pixel_values.to(device)

def predict_image(image, model, feature_extractor):
    inputs = transform(image)
    with torch.no_grad():
        outputs = model(inputs)
    return np.argmax(outputs[0].cpu().numpy())

In [13]:
label_counts = {}

In [14]:
def process_row(row):
    image_url = row['image_url']
    predicted_label = -1  # default value for when the image cannot be classified
    try:
        response = requests.get(image_url, headers={'User-Agent': 'Mozilla/5.0'})
        image_name = os.path.basename(image_url)
        mime_type = image_url.split('.')[-1]
        image_path = os.path.join(folder_path, f'train-0-{row.name}.{mime_type}')
        with open(image_path, 'wb') as f:
            f.write(response.content)
        predicted_label = predict_image(image_path, model, feature_extractor)
        if predicted_label not in label_counts:
            label_counts[predicted_label] = 0
        label_counts[predicted_label] += 1
        if predicted_label == 2:
            os.remove(image_path)
    except Exception as e:
        print(f"Exception is {e} for {image_url}")
        if image_path:
            os.remove(image_path)
            
last_index_processed = None

# Create a thread pool with 10 threads
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_row, row) for _, row in df.iloc[61265:].iterrows()]
    for future in as_completed(futures):
        if future.exception() is not None:
            print(f"Exception occurred: {future.exception()}")
        else:
            last_index_processed = future.result().name
            if last_index_processed%100 == 0:
                print(f"{str(last_index_processed)} done")
                

Exception is cannot identify image file 'images/train-0-61332.jpg' for https://upload.wikimedia.org/wikipedia/commons/c/c6/Cairo_Stadium.jpg
Exception is cannot identify image file 'images/train-0-61436.jpg' for https://upload.wikimedia.org/wikipedia/commons/9/9d/Yaghnobi_speaking_areas.jpg
Exception is cannot identify image file 'images/train-0-61581.jpg' for https://upload.wikimedia.org/wikipedia/en/4/4b/R.K._Jackson_Photo_1.jpg
Exception is Truncated sRGB chunk for https://upload.wikimedia.org/wikipedia/commons/f/fb/Signal_transduction_pathways.png
Exception is cannot identify image file 'images/train-0-61620.jpg' for https://upload.wikimedia.org/wikipedia/commons/8/87/1971_-_Garden_Party_I_Poster_%282%29.jpg
Exception is cannot identify image file 'images/train-0-61653.jpg' for https://upload.wikimedia.org/wikipedia/commons/2/20/Storks_samarkand.jpg
Exception is cannot identify image file 'images/train-0-61654.jpg' for https://upload.wikimedia.org/wikipedia/commons/1/10/Francis_Mos

/opt/homebrew/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Exception is cannot identify image file 'images/train-0-62314.JPG' for https://upload.wikimedia.org/wikipedia/commons/0/00/Brasao_de_acailandia.JPG
Exception is cannot identify image file 'images/train-0-62363.png' for https://upload.wikimedia.org/wikipedia/commons/4/4f/Distribution_of_Hypericum_seniawinii.png
Exception is cannot identify image file 'images/train-0-62420.jpg' for https://upload.wikimedia.org/wikipedia/commons/e/ed/Kigali_Arena.jpg
Exception is cannot identify image file 'images/train-0-62482.jpg' for https://upload.wikimedia.org/wikipedia/commons/6/6f/Hayat_Baksh_%28Bestower_of_life%29_terrace.jpg
Exception is cannot identify image file 'images/train-0-62532.jpg' for https://upload.wikimedia.org/wikipedia/commons/f/f0/Wiki79.jpg
Exception is cannot identify image file 'images/train-0-62543.png' for https://upload.wikimedia.org/wikipedia/commons/1/18/Wiki_plot_03.png
Exception is cannot identify image file 'images/train-0-62582.jpg' for https://upload.wikimedia.org/wiki

/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (137881212 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Exception is cannot identify image file 'images/train-0-63891.png' for https://upload.wikimedia.org/wikipedia/commons/e/e0/Cheyenne-CARRON.png
Exception is cannot identify image file 'images/train-0-64054.png' for https://upload.wikimedia.org/wikipedia/en/2/2a/Hudson_-_Bahrain.png
Exception is cannot identify image file 'images/train-0-64281.jpg' for https://upload.wikimedia.org/wikipedia/commons/4/4d/Sambazon_Inc._Logo.jpg
Exception is cannot identify image file 'images/train-0-64283.jpg' for https://upload.wikimedia.org/wikipedia/commons/4/44/%D8%AC%D9%88%D8%B1%D8%AC_%D9%85%D8%B1%D8%B2%D8%A8%D8%A7%D8%AE.jpg
Exception is cannot identify image file 'images/train-0-64311.png' for https://upload.wikimedia.org/wikipedia/commons/8/80/Townshend_International_School_Logo.png
Exception is cannot identify image file 'images/train-0-64352.jpg' for https://upload.wikimedia.org/wikipedia/commons/a/a6/Erstes_essbares_Foto_der_Welt%2C_%C2%A9_1976_Josef_H._Neumann.jpg
Exception is cannot identify im

/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (99207936 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
predictions = []
for index, row in df.iloc[19606:].iterrows():
    if index%100==0:
        print(f"{str(index)} done!")
        gc.collect()
    image_url = row['image_url']
    predicted_label = -1  # default value for when the image cannot be classified at all
    try:
        response = requests.get(image_url, headers={'User-Agent': 'Mozilla/5.0'})
        image_name = os.path.basename(image_url)
        mime_type = image_url.split('.')[-1]
        image_path = os.path.join(folder_path, f'train-0-{index}.{mime_type}')
        with open(image_path, 'wb') as f:
            f.write(response.content)
        predicted_label = predict_image(image_path, model, feature_extractor)
        if predicted_label not in label_counts:
            label_counts[predicted_label] = 0
        label_counts[predicted_label] += 1
        if predicted_label == 2:
            os.remove(image_path)
    except Exception as e:
        print(f"Exception is {e} for {image_url}")
        if image_path:
            os.remove(image_path)

In [ ]:
label_counts

In [ ]:
gc.collect()

In [ ]:
index

In [ ]:
for p in predictions:
    if p['label'] not in label_counts:
          label_counts[p['label']] = 0
    label_counts[p['label']] += 1

for label, count in label_counts.items():
    print(f"Label {label}: {count} predictions")

if not os.path.exists('not_label_2_new'):
    os.makedirs('not_label_2_new')

# Loop through the predictions and copy the images with label not 2 (others) to the new folder
for p in predictions:
    if p['label'] != 2:
        image_name = os.path.basename(p['image'])
        image_path = os.path.join(test_folder, image_name)
        shutil.copy(image_path, 'not_label_2_new')